Testing DB connection

In [4]:
import pyodbc
import requests

In [2]:
server = 'DA_Shared_SP_4'  # Replace with the name of your server
database = 'DT_RA_BufferTesting_2022'  # Replace with the name of your database

# Establish the connection using Windows authentication
conn = pyodbc.connect('Driver={SQL Server};'
                      'Server=' + server + ';'
                      'Database=' + database + ';'
                      'Trusted_Connection=yes;')

In [3]:
cursor = conn.cursor()
cursor.execute("SELECT TABLE_NAME FROM INFORMATION_SCHEMA.TABLES WHERE TABLE_TYPE='BASE TABLE';")

# Fetch all the table names
table_names = cursor.fetchall()

# Print the table names
for table in table_names:
    print(table[0])

RAW_HPSS_Rates
RAW_Emp_Test_Data
REF_Award_Rates_HPSS
REF_InstrumentRates
RAW_BFI_Award_Rates
REF_BFI_Award_Rates
REF_BFI_Test_Table
REF_Buffer_Emp_Data


In [5]:
# CoinGecko API endpoint for historical Bitcoin prices
url = 'https://api.coingecko.com/api/v3/coins/bitcoin/market_chart'
params = {
    'vs_currency': 'usd',
    'days': '1',
}

# Establish connection to Azure SQL Database
cursor = conn.cursor()

# Create Bitcoin_Daily_Prices table if it doesn't exist
create_table_query = '''
    IF NOT EXISTS (SELECT * FROM sys.tables WHERE name='Bitcoin_Daily_Prices')
    BEGIN
        CREATE TABLE Bitcoin_Daily_Prices (
            [Date] DATE,
            [Price] FLOAT
        )
    END
'''
cursor.execute(create_table_query)
conn.commit()

# Retrieve Bitcoin closing prices
response = requests.get(url, params=params)
if response.status_code == 200:
    data = response.json()
    timestamps = data['prices']
    for timestamp, price in timestamps:
        # Format the timestamp and price as per your database schema
        date = datetime.datetime.fromtimestamp(timestamp // 1000).date()  # Convert from milliseconds to date
        check_duplicate_query = f"SELECT COUNT(*) FROM Bitcoin_Daily_Prices WHERE [Date] = ?"
        cursor.execute(check_duplicate_query, date)
        count = cursor.fetchone()[0]
        if count == 0:
            insert_query = f"INSERT INTO Bitcoin_Daily_Prices ([Date], [Price]) VALUES (?, ?)"
            cursor.execute(insert_query, date, price)
    conn.commit()
    print("Data loaded successfully!")
else:
    print("Failed to retrieve data from the API.")

# Close the database connection
cursor.close()
conn.close()

ConnectionError: HTTPSConnectionPool(host='api.coingecko.com', port=443): Max retries exceeded with url: /api/v3/coins/bitcoin/market_chart?vs_currency=usd&days=1 (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x0000022B5159D640>: Failed to establish a new connection: [WinError 10060] A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond'))